# The Price to Sales Ratio (P/S) model

This Notebook will be used for calculating the Price to Sales (P/S) model as described in the Medium Post [Sure you know the stock price. But do you know its value?](https://medium.com/@sankha.mukherjee_007/sure-you-know-the-stock-price-but-do-you-know-its-value-65ab44dadd04) in Section 4.3. 

This implementation will be as close as possible to the original article so that one is able to easily follow the original article and this one.

In [1]:
import financeMacroFactors as fM
import pandas as pd
import numpy as np
from scipy import interpolate
from datetime import datetime as dt
from datetime import timedelta as tDel
import matplotlib.pyplot as plt

In [2]:
fundamentals = fM.companies.getTickerFundamentalDataMW('MSFT')

## Download & Calculate Fundamental Data

Before we do the calculations, we should download and calculate the fundamental data that we need. Here as well, we shall follow the article as closely as possible.

### Download the revenue

The revenue $\mathbf{r}$ will be placed in the Python variable `p`:

In [3]:
revenue = fM.companies.extractYearlyData(fundamentals['IncomeStatement'], 'Sales/Revenue')
y, r = zip(*revenue)
r = np.array(r)
print(f'--------[Revenue]------------')
print(f'Years: {[m.year for m in y]}')
print(f'Revenue during that time: r = {r}')

--------[Revenue]------------
Years: [2016, 2017, 2018, 2019, 2020]
Revenue during that time: r = [8.4700e+10 9.6020e+10 1.1018e+11 1.2550e+11 1.4302e+11]


### Download the total number of outstanding shares

The total number of outstanding shares $\mathbf{s}$ is placed in the python variable `s`:

In [4]:
shares = fM.companies.extractYearlyData(fundamentals['IncomeStatement'], 'Diluted Shares Outstanding')
shares
ya, s = zip(*shares)
s = np.array(s)
print(f'--------[Diluted shares Outstanding]------------')
print(f'Years: {[m.year for m in ya]}')
print(f'Diluted shares outstanding: s = {s}')

--------[Diluted shares Outstanding]------------
Years: [2016, 2017, 2018, 2019, 2020]
Diluted shares outstanding: s = [8.01e+09 7.83e+09 7.79e+09 7.75e+09 7.68e+09]


### Download the stock price 

The stock proce $\mathbf{s}$ is placed in the python variable `p`. Note that the stock proce has two properties that one needs to take care of:

1. Monthly-averaged stock price is averaged on the first of the month, while the fundamental data is available at the end of the month. For this reason, the share price will have to be typically shifted by a day to match the dates of the fundamental data. The averaged stock price data is for the first of the *next month*.
2. The stock data is arranged such that the dates are descending, while the fundamental data is arranged such that the dates are ascending. To fix this problem. the stock data is reversed.


In [5]:
stockPrice = fM.companies.yahooData.getStockDataYahoo('msft', y[0]-tDel(90), dt.now(), '1mo')

In [6]:
stockPrice = stockPrice[1:]
p = np.array([m[-2] for m in stockPrice if ((m[0]-tDel(1)) in y)])
p = p[::-1]
print(f'Stock Price over the same period = {p}')

Stock Price over the same period = [ 52.6   69.15 102.87 134.29 204.52]


## Calculate the P/S Ratio

This is the stock price ($\mathbf{p}$) divided by the revenue/stock ( $\mathbf r$/$\mathbf s$ ). Note that all operations are element-wise operations. From this point on, there are only two more operations to perform:

### Calculate the P/S ratio over the last few years


In [7]:
ps = p*s/r
ps

array([ 4.97433294,  5.63887211,  7.27316482,  8.29280876, 10.98247518])

### Use the average P/S ratio to claculate the P/S valuation

In [8]:
ps.mean()*((r/s)[-1])

138.40780542987875